In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

# Arguments

In [2]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [3]:
pretrained = './log/style1/128/checkpoint.pth.tar'
resume = ''

In [4]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 5000
start_epoch = 0
train_batch = 32
test_batch = 300
lr = 0.001
schedule = [600]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/to_pggan/128_noaug' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

best_acc = 0

In [5]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [6]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
#     transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes, override_params={'dropout_rate':0.0})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/checkpoint.pth.tar'


In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.01M


In [10]:
# summary(model, input_size=(3,128,128),device='cuda')

# Model tuning

In [11]:
for i, param in enumerate(model.parameters()):
    if i >= 209:
        break
    param.requires_grad = False

In [12]:
summary(model, input_size=(3,128,128),device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 129, 129]               0
Conv2dStaticSamePadding-2           [-1, 32, 64, 64]             864
         GroupNorm-3           [-1, 32, 64, 64]              64
MemoryEfficientSwish-4           [-1, 32, 64, 64]               0
         ZeroPad2d-5           [-1, 32, 66, 66]               0
Conv2dStaticSamePadding-6           [-1, 32, 64, 64]             288
         GroupNorm-7           [-1, 32, 64, 64]              64
MemoryEfficientSwish-8           [-1, 32, 64, 64]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 32, 1, 1]             288
         I

# Loss

In [13]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=0.01, nesterov=True)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [14]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [15]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [16]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [18]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        test_loss, test_acc = test(val_target_loader, model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, model, criterion, epoch, use_cuda)

        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        scheduler_warmup.step()

        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)


Epoch: [1 | 5000] LR: 0.001000
1/7 | Total:0:00:01 | ETA:0:00:08 | Loss:4.579432964324951 | top1:46.875
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:4.194717884063721 | top1:50.0


/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


107/107 | Total:0:00:35 | ETA:0:00:00 | Loss:4.162331690298062 | top1:49.98754119873047
26/26 | Total:0:00:17 | ETA:0:00:00 | Loss:0.0038308648710461478 | top1:99.88461303710938

Epoch: [2 | 5000] LR: 0.001700
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:3.8245632648468018 | top1:53.125
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:4.0104935963948565 | top1:49.47916793823242

Epoch: [3 | 5000] LR: 0.001700
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:3.345350503921509 | top1:56.25
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:3.5141841570536294 | top1:51.04166793823242

Epoch: [4 | 5000] LR: 0.001700
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:3.6907780170440674 | top1:43.75
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:3.1397668917973838 | top1:50.0

Epoch: [5 | 5000] LR: 0.001700
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:2.876948356628418 | top1:50.0
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:2.678878386815389 | top1:50.520835876464844

Epoch: [6 | 5000] LR: 0.001700
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:2.50

1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.7133533358573914 | top1:34.375
7/7 | Total:0:00:00 | ETA:0:00:01 | Loss:0.7244923412799835 | top1:38.020835876464844

Epoch: [46 | 5000] LR: 0.001700
1/7 | Total:0:00:00 | ETA:0:00:03 | Loss:0.6894612312316895 | top1:46.875
7/7 | Total:0:00:00 | ETA:0:00:01 | Loss:0.7219007213910421 | top1:40.10416793823242

Epoch: [47 | 5000] LR: 0.001700
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.7024272084236145 | top1:43.75
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.7226273616154989 | top1:38.020835876464844

Epoch: [48 | 5000] LR: 0.001700
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.7052611708641052 | top1:46.875
7/7 | Total:0:00:00 | ETA:0:00:01 | Loss:0.7197453478972117 | top1:40.625

Epoch: [49 | 5000] LR: 0.001700
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.7140191793441772 | top1:43.75
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.7224332888921102 | top1:39.583335876464844

Epoch: [50 | 5000] LR: 0.001700
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.7344785

1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6941826939582825 | top1:50.0
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.7118423779805502 | top1:39.0625

Epoch: [89 | 5000] LR: 0.002400
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.6878053545951843 | top1:71.875
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.7155726750691732 | top1:40.10416793823242

Epoch: [90 | 5000] LR: 0.002400
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.7187105417251587 | top1:37.5
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.7173828482627869 | top1:38.020835876464844

Epoch: [91 | 5000] LR: 0.002400
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.7292979955673218 | top1:25.0
7/7 | Total:0:00:00 | ETA:0:00:01 | Loss:0.7135796745618185 | top1:40.10416793823242

Epoch: [92 | 5000] LR: 0.002400
1/7 | Total:0:00:00 | ETA:0:00:03 | Loss:0.6965556740760803 | top1:46.875
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.7148317794005076 | top1:39.583335876464844

Epoch: [93 | 5000] LR: 0.002400
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.74788308143

...

1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6928600072860718 | top1:59.375
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6929067273934683 | top1:60.9375

Epoch: [4791 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6926434636116028 | top1:59.375
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6928818821907043 | top1:58.85416793823242

Epoch: [4792 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6925478577613831 | top1:59.375
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6928758223851522 | top1:59.895835876464844

Epoch: [4793 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6929764747619629 | top1:62.5
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6929058531920115 | top1:61.97916793823242

Epoch: [4794 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6931724548339844 | top1:46.875
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6928701102733612 | top1:58.85416793823242

Epoch: [4795 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:

7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6929139792919159 | top1:53.645835876464844

Epoch: [4833 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.69315105676651 | top1:50.0
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6928869386514028 | top1:53.125

Epoch: [4834 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6931780576705933 | top1:46.875
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6929360926151276 | top1:50.520835876464844

Epoch: [4835 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6927570104598999 | top1:59.375
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6928705473740896 | top1:59.375

Epoch: [4836 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.692620038986206 | top1:62.5
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6928686996301016 | top1:52.083335876464844

Epoch: [4837 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6931169629096985 | top1:43.75
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.69291030367215

1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6929839849472046 | top1:56.25
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6929277380307516 | top1:52.083335876464844

Epoch: [4876 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6926488876342773 | top1:59.375
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.692888488372167 | top1:54.6875

Epoch: [4877 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.6929206252098083 | top1:53.125
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6929066677888235 | top1:51.5625

Epoch: [4878 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.6927845478057861 | top1:46.875
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.692873626947403 | top1:52.60416793823242

Epoch: [4879 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6927928924560547 | top1:43.75
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6929177641868591 | top1:54.16666793823242

Epoch: [4880 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.6928385496

7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6929464538892111 | top1:57.29166793823242

Epoch: [4918 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6926971077919006 | top1:62.5
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6928926209608713 | top1:59.375

Epoch: [4919 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6928458213806152 | top1:56.25
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6928970118363699 | top1:56.25

Epoch: [4920 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6930117607116699 | top1:68.75
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6928474605083466 | top1:62.5

Epoch: [4921 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.6924856305122375 | top1:68.75
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6929108798503876 | top1:59.375

Epoch: [4922 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6928920149803162 | top1:50.0
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6929044425487518 | top1:59.375

Epoch: [4

1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.693074643611908 | top1:56.25
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6928951938947042 | top1:50.0

Epoch: [4961 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6928838491439819 | top1:53.125
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6929051776727041 | top1:49.47916793823242

Epoch: [4962 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.692943811416626 | top1:40.625
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6928858458995819 | top1:50.520835876464844

Epoch: [4963 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6928972005844116 | top1:56.25
7/7 | Total:0:00:00 | ETA:0:00:01 | Loss:0.6929212609926859 | top1:50.0

Epoch: [4964 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6930347084999084 | top1:56.25
7/7 | Total:0:00:00 | ETA:0:00:01 | Loss:0.692898670832316 | top1:51.04166793823242

Epoch: [4965 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:04 | Loss:0.6928243637084961 |